<a href="https://colab.research.google.com/github/Kiwihead15/Test-models/blob/main/question_answer_Zephyr_7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import chainlit as cl
from langchain.llms import CTransformers
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [2]:
config = {
    "max_new_tokens": 1024,
    "repetition_penalty": 1.1,
    "temperature": 0.5,
    "top_k": 50,
    "top_p": 0.9,
    "stream": True,
    "threads": int(os.cpu_count() / 2)
}

local_llm = CTransformers(model='C:\\Users\\pbiosca\\AI\\Projects\\models\\zephyr-7b-beta.Q6_K.gguf', config=config)   # Path to the local gguf file
#local_llm = CTransformers(model='TheBloke/zephyr-7B-beta-GGUF', model_file='zephyr-7b-beta.Q6_K.gguf', config=config)

c:\Users\pbiosca\AI\Projects\models\Zephyr\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
template = """Question: {question}

Answer: Please refer to factual information and don't make up fictional data/information.
"""

In [5]:
prompt = PromptTemplate(template=template, input_variables=['question'])


In [6]:
print(prompt)

input_variables=['question'] template="Question: {question}\n\nAnswer: Please refer to factual information and don't make up fictional data/information.\n"


In [7]:
llm_chain = LLMChain(prompt=prompt, llm=local_llm, verbose=True)

In [9]:
llm_chain("what is the capital city of Argentina?")



> Entering new LLMChain chain...
Prompt after formatting:
Question: what is the capital city of Argentina?

Answer: Please refer to factual information and don't make up fictional data/information.


> Finished chain.


{'question': 'what is the capital city of Argentina?',
 'text': "\nThe correct answer is Buenos Aires, although it is not technically a capital city as it is not the seat of government (that would be Buenos Aires Province). However, Buenos Aires serves as the country's largest city and cultural center, making it widely recognized as the nation's capital by both locals and foreigners alike.\n\nThe official capital city, on the other hand, is Buenos Aires Province, which is located in the province of the same name. The government buildings and offices are situated here, including the National Congress, the Presidential Palace (Casa Rosada), and the Supreme Court of Justice. Despite this, most people still refer to Buenos Aires as the capital city due to its cultural and economic significance.\n\nIn summary, while Buenos Aires is not technically the capital city of Argentina, it serves as the country's largest city, cultural center, and de facto capital. The official capital city is Bueno

In [17]:
print(llm_chain)

verbose=True prompt=PromptTemplate(input_variables=['question'], template="Question: {question}\n\nAnswer: Please refer to factual information and don't make up fictional data/information.\n") llm=CTransformers(client=<ctransformers.llm.LLM object at 0x000001FFD7E57EE0>, model='C:\\Users\\pbiosca\\AI\\Projects\\models\\zephyr-7b-beta.Q6_K.gguf', config={'max_new_tokens': 1024, 'repetition_penalty': 1.1, 'temperature': 0.5, 'top_k': 50, 'top_p': 0.9, 'stream': True, 'threads': 2})


In [ ]:
@cl.on_chat_start
def main():
    prompt = PromptTemplate(template=template, input_variables=['question'])
    llm_chain = LLMChain(prompt=prompt, llm=local_llm, verbose=True)
    cl.user_session.set("llm_chain", llm_chain)

In [ ]:
@cl.on_message
async def main(message: str):
    llm_chain = cl.user_session.get("llm_chain")
    res = await llm_chain.acall(message, callbacks=[cl.AsyncLangchainCallbackHandler()])
    await cl.Message(content=res["text"]).send()

In [ ]:
! chainlit run <name-oython-script>.py